In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import io

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving genre_df_reference to genre_df_reference


In [ ]:
genre_df_with_comments = pd.read_csv(io.BytesIO(uploaded['genre_df_reference']),index_col=0)

In [ ]:
 genre_df_with_comments.head()

,subreddit,title,comment,author,created_at,genre,sentiment score
0,Music,samrat goswami - my heart [pop] hey everyone! ...,NaN,IntrepidCucumber1902,2022-12-31 03:28:53,pop,0.296875
1,Music,why u2 is so hated (my take),i know there's a topic about that every month ...,Expensive_Ad_403,2022-12-31 02:57:06,pop,0.132143
2,Music,new covers! pop/indie covers,[removed],jamiesomervillecom,2022-12-31 02:33:26,pop,0.000000
3,Music,the cult’s ‘under the midnight sun’ is one of ...,like a lot of bands and musicians who released...,The_Norfolkian,2022-12-31 00:56:56,pop,0.198958
4,Music,official top pop songs of 2022 https://7r6.com...,NaN,SentenceNo8256,2022-12-31 00:26:21,pop,0.500000


In [ ]:
word_freq_comment = pd.Series(' '.join(genre_df_with_comments['comment'].astype(str)).split()).value_counts()
word_freq_title = pd.Series(' '.join(genre_df_with_comments['title'].astype(str)).split()).value_counts()


In [ ]:
combined_word_freq = pd.concat([word_freq_comment, word_freq_title], axis=1, keys=['comment', 'title']).fillna(0)
combined_word_freq['total'] = combined_word_freq.sum(axis=1)
word_freq = combined_word_freq['total']

In [ ]:
word_freq_dict = word_freq.to_dict()

In [ ]:
!pip install bertopic
!pip install bertopic[flair]
!pip install bertopic[gensim]
!pip install bertopic[spacy]
!pip install bertopic[use]

In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2", top_n_words=15, verbose=True, calculate_probabilities=True, nr_topics="auto")
topic_model.save("genre_topics")

In [ ]:
genre_df_with_comments.head()

,subreddit,title,comment,author,created_at,genre,sentiment score
0,Music,samrat goswami - my heart [pop] hey everyone! ...,NaN,IntrepidCucumber1902,2022-12-31 03:28:53,pop,0.296875
1,Music,why u2 is so hated (my take),i know there's a topic about that every month ...,Expensive_Ad_403,2022-12-31 02:57:06,pop,0.132143
2,Music,new covers! pop/indie covers,[removed],jamiesomervillecom,2022-12-31 02:33:26,pop,0.000000
3,Music,the cult’s ‘under the midnight sun’ is one of ...,like a lot of bands and musicians who released...,The_Norfolkian,2022-12-31 00:56:56,pop,0.198958
4,Music,official top pop songs of 2022 https://7r6.com...,NaN,SentenceNo8256,2022-12-31 00:26:21,pop,0.500000


In [ ]:
genre_topics_pop = genre_df_with_comments[(genre_df_with_comments['genre']=='pop')]
genre_topics_pop  = genre_topics_pop['comment'].dropna().astype(str).reset_index(drop = True)

genre_topics_rock = genre_df_with_comments[(genre_df_with_comments['genre']=='rock')]
genre_topics_rock  = genre_topics_rock['comment'].dropna().astype(str).reset_index(drop = True)

In [ ]:
topics, probs = topic_model.fit_transform(genre_topics_pop)

Batches:   0%|          | 0/78 [00:00<?, ?it/s]

2023-04-28 04:07:31,056 - BERTopic - Transformed documents to Embeddings
2023-04-28 04:07:45,773 - BERTopic - Reduced dimensionality
2023-04-28 04:07:46,291 - BERTopic - Clustered reduced embeddings
2023-04-28 04:07:51,271 - BERTopic - Reduced number of topics from 56 to 56


In [ ]:
topic_model.get_document_info(genre_topics_pop)

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,i know there's a topic about that every month ...,-1,-1_the_and_of_it,the - and - of - it - to - that - in - pop - i...,0.378189,False
1,[removed],6,6_removed_deleted__,removed - deleted - - - - - - - - - -...,0.156107,True
2,like a lot of bands and musicians who released...,4,4_they_their_band_album,they - their - band - album - were - the - the...,0.092055,True
3,hello people i'm danny and i'd like to share m...,-1,-1_the_and_of_it,the - and - of - it - to - that - in - pop - i...,0.774230,False
4,my friend released her first song!! honestly y...,1,1_she_her_uchis_the,she - her - uchis - the - of - and - was - in ...,0.110054,False
...,...,...,...,...,...,...
2470,linda was the highest paid performer several y...,-1,-1_the_and_of_it,the - and - of - it - to - that - in - pop - i...,0.422228,False
2471,"i think the '70s were peak music, all the musi...",11,11_duran_and_music_like,duran - and - music - like - to - was - of - t...,0.014000,False
2472,* violins - lagwagon\n* heaven is falling - ba...,16,16_by_neo_jazz_psychedelia,by - neo - jazz - psychedelia - hop - foot - 1...,0.923505,False
2473,"* fred hersch, esperanza spalding - alive at v...",16,16_by_neo_jazz_psychedelia,by - neo - jazz - psychedelia - hop - foot - 1...,0.756904,True


In [ ]:
topic_model.visualize_barchart(n_words=15, width=400, height=400, top_n_topics=44)

In [ ]:
topics, probs = topic_model.fit_transform(genre_topics_rock)

Batches:   0%|          | 0/72 [00:00<?, ?it/s]

2023-04-28 04:10:34,335 - BERTopic - Transformed documents to Embeddings
2023-04-28 04:10:48,164 - BERTopic - Reduced dimensionality
2023-04-28 04:10:48,537 - BERTopic - Clustered reduced embeddings
2023-04-28 04:10:51,624 - BERTopic - Reduced number of topics from 42 to 25


In [ ]:
topic_model.get_document_info(genre_topics_rock)

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,"i know it's kinda an old topic, but i have som...",-1,-1_and_the_of_to,and - the - of - to - rock - in - that - is - ...,0.482595,False
1,bo diddley's birthday is december 30. he's one...,-1,-1_and_the_of_to,and - the - of - to - rock - in - that - is - ...,0.454051,False
2,the documentary was a real eye opener. i grew ...,0,0_the_it_and_of,the - it - and - of - to - but - is - that - a...,1.000000,False
3,i don't know if this counts as a playlist post...,0,0_the_it_and_of,the - it - and - of - to - but - is - that - a...,0.678554,False
4,watching the hbo 2022 hall of fame special. em...,2,2_fame_hall_roll_inducted,fame - hall - roll - inducted - and - of - roc...,0.210020,False
...,...,...,...,...,...,...
2280,rock the blues away by ac / dc,-1,-1_and_the_of_to,and - the - of - to - rock - in - that - is - ...,0.443653,False
2281,you should check out the howlin’ wolf album an...,-1,-1_and_the_of_to,and - the - of - to - rock - in - that - is - ...,0.818929,False
2282,i think it’s a lack of rhythm and blues influe...,3,3_folk_rock_country_blues,folk - rock - country - blues - and - amp - mu...,1.000000,False
2283,redditors when anything besides old rock or bl...,-1,-1_and_the_of_to,and - the - of - to - rock - in - that - is - ...,0.657297,False


In [ ]:
topic_model.visualize_barchart(n_words=15, width=400, height=400, top_n_topics=44)

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
  print(topic_model.get_topic_info())

In [ ]:
for i in range(0,len(topic_model.get_topic_info())-1):
  print(topic_model.get_topic(i))